# Tutorial 3: How simulations define your predictions
The inverse problem has no unique solution as it is ill-posed. In order to solve it we need to constraint the space of possible solutions. While inverse solutions like minimum-norm estimates have an explicit constraint of minimum-energy, the constraints with esinet are implicit and mostly shaped by the simulations.

This tutorial aims the relation between simulation parameters and predictions.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import mne
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info
plot_params = dict(surface='white', hemi='both', verbose=0)

## Create Forward model
First we create a template forward model which comes with the esinet package

In [2]:
info = get_info(sfreq=100)
fwd = create_forward_model(info=info)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  14 | elapsed:    1.1s remaining:    2.6s
[Parallel(n_jobs=-1)]: Done   7 out of  14 | elapsed:    1.1s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  10 out of  14 | elapsed:    1.1s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  14 out of  14 | elapsed:    1.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  14 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   7 out of  14 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  10 out of  14 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  14 out of  14 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  14 | elapsed:    0.5s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done   7 out of  14 | elapsed:    0.5

## Simulate
Next, we simulate two types of data: 
1. Data containing small sources with 15-25 mm in diameter.
2. Data containing large sources with 35-45 mm in diameter.

Note, that for publication-ready inverse solutions you should increase the number of training samples to 100,000.

In [3]:
n_samples = 100
settings_small = dict(number_of_sources=(1, 10), extents=(15, 25))
settings_large = dict(number_of_sources=(1, 10), extents=(35, 45))

sim_small = Simulation(fwd, info, settings=settings_small).simulate(n_samples=n_samples)
sim_large = Simulation(fwd, info, settings=settings_large).simulate(n_samples=n_samples)


Simulating data based on sparse patches.


100%|██████████| 100/100 [00:01<00:00, 59.09it/s]


Simulating data based on sparse patches.


100%|██████████| 100/100 [00:01<00:00, 59.04it/s]


## Lets visualize the two types of simulations
The two brain plots should now look quite different, as one contains large and extended sources while the other contains tiny point-like sources.

In [5]:
idx = 0
brain = sim_small.source_data[idx].plot(**plot_params)
brain.add_text(0.1, 0.9, 'Small sources', 'title',
               font_size=14)

brain = sim_large.source_data[idx].plot(**plot_params)
brain.add_text(0.1, 0.9, 'Large sources', 'title',
               font_size=14)

## Train individual neural networks

In [4]:
model_type = 'ConvDip'  # can be 'FC' or 'ConvDip', too
net_small = Net(fwd, verbose=True, model_type=model_type).fit(sim_small, epochs=10)
net_large = Net(fwd, verbose=True, model_type=model_type).fit(sim_large, epochs=10)

preprocess data


Model: "ConvDip_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input_ConvDip (InputLayer)      │ (None, None, 9, 9, 1)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, None, 7, 7, 64) │           640 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Drop_conv2d_0 (Dropout)         │ (None, None, 7, 7, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, None, 5, 5, 64) │        36,928 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Drop_conv2d_1 (Dropout)         │ (None, None, 5, 5, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, None, 1600)     │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, None, 200)      │       320,200 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Drop_FC_0 (Dropout)             │ (None, None, 200)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC_Out (TimeDistributed)        │ (None, None, 1284)     │       258,084 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 615,852 (2.35 MB)

 Trainable params: 615,852 (2.35 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input_ConvDip (InputLayer)      │ (None, None, 9, 9, 1)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, None, 7, 7, 64) │           640 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Drop_conv2d_0 (Dropout)         │ (None, None, 7, 7, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, None, 5, 5, 64) │        36,928 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Drop_conv2d_1 (Dropout)         │ (None, None, 5, 5, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, None, 1600)     │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_7              │ (None, None, 200)      │       320,200 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Drop_FC_0 (Dropout)             │ (None, None, 200)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC_Out (TimeDistributed)        │ (None, None, 1284)     │       258,084 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 615,852 (2.35 MB)

 Trainable params: 615,852 (2.35 MB)

 Non-trainable params: 0 (0.00 B)

	...done
fit model
Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 22s 505ms/step - loss: -0.0230 - val_loss: -0.0780
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 198ms/step - loss: -0.1263 - val_loss: -0.0968
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 189ms/step - loss: -0.1697 - val_loss: -0.1148
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 179ms/step - loss: -0.2089 - val_loss: -0.1440
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 190ms/step - loss: -0.2293 - val_loss: -0.1587
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 186ms/step - loss: -0.2476 - val_loss: -0.1982
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 183ms/step - loss: -0.2651 - val_loss: -0.2090
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 185ms/step - loss: -0.2858 - val_loss: -0.2091
Epoch 9/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 195ms/step - loss: -0.2804 - val_loss: -0.2129
Epoch 10/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 217ms/step - loss: -0.3193 - val_loss: -0.2246


Now we have simulated two different types of source & eeg data and build two neural networks that each was trained on one of these simulations. Lets see how they perform within their own simulation type.

In [ ]:
# Simulate some new, unseen test data    
n_test_samples = 2
sim_test_small = Simulation(fwd, info, settings=settings_small).simulate(n_samples=n_test_samples)
sim_test_large = Simulation(fwd, info, settings=settings_large).simulate(n_samples=n_test_samples)


brain = sim_test_small.source_data[0].plot(**plot_params)
brain.add_text(0.1, 0.9, 'Ground Truth of small data', 'title',
               font_size=14)


brain = net_small.predict(sim_test_small)[0].plot(**plot_params)
brain.add_text(0.1, 0.9, 'Small-Net on small data', 'title',
               font_size=14)



brain = sim_test_large.source_data[0].plot(**plot_params)
brain.add_text(0.1, 0.9, 'Ground Truth of large data', 'title',
               font_size=14)


brain = net_large.predict(sim_test_large)[0].plot(**plot_params)
brain.add_text(0.1, 0.9, 'Large-Net on large data', 'title',
               font_size=14)

Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 38.09it/s]


Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 25.18it/s]


interpolating for convdip...


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step



/Users/zacariabalkhy/UCDAVIS_COURSES/MAE298/final_project/esinet_MAE298/tutorials/../esinet/net.py:1078: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  base_scaler = rms_true / rms_est


Residual Variance(s): [13.42, 14.13] [%]


/Users/zacariabalkhy/UCDAVIS_COURSES/MAE298/final_project/esinet_MAE298/venv/lib/python3.12/site-packages/ipykernel/eventloops.py:145: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  el.exec() if hasattr(el, "exec") else el.exec_()
/Users/zacariabalkhy/UCDAVIS_COURSES/MAE298/final_project/esinet_MAE298/venv/lib/python3.12/site-packages/matplotlib/layout_engine.py:271: RuntimeWarning: divide by zero encountered in scalar divide
  h_pad = self._params['h_pad'] / height
/Users/zacariabalkhy/UCDAVIS_COURSES/MAE298/final_project/esinet_MAE298/venv/lib/python3.12/site-packages/ipykernel/eventloops.py:145: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  el.exec() if hasattr(el, "exec") else el.exec_()


: 

Now we will use the large-net to predict the small simulation and vice versa.

In [ ]:
brain = sim_test_small.source_data[0].plot(**plot_params)
brain.add_text(0.1, 0.9, 'Ground Truth of small data', 'title',
               font_size=14)


brain = net_large.predict(sim_test_small)[0].plot(**plot_params)
brain.add_text(0.1, 0.9, 'Large-Net on small data', 'title',
               font_size=14)



brain = sim_test_large.source_data[0].plot(**plot_params)
brain.add_text(0.1, 0.9, 'Ground Truth of large data', 'title',
               font_size=14)


brain = net_small.predict(sim_test_large)[0].plot(**plot_params)
brain.add_text(0.1, 0.9, 'Small-Net on large data', 'title',
               font_size=14)

interpolating for convdip...


2it [00:00, 21.67it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step



/Users/zacariabalkhy/UCDAVIS_COURSES/MAE298/final_project/esinet_MAE298/tutorials/../esinet/net.py:1081: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  opt = minimize_scalar(self.correlation_criterion, args=(self.leadfield, y_est* base_scaler, x_true), \


Residual Variance(s): [10.27, 16.34] [%]
interpolating for convdip...


2it [00:00, 23.19it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step



/Users/zacariabalkhy/UCDAVIS_COURSES/MAE298/final_project/esinet_MAE298/tutorials/../esinet/net.py:1081: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  opt = minimize_scalar(self.correlation_criterion, args=(self.leadfield, y_est* base_scaler, x_true), \


Residual Variance(s): [15.64, 16.85] [%]


Using control points [8.77144155e-10 1.00001300e-09 1.39356736e-09]
Reading labels from parcellation...
   read 35 labels from /Users/zacariabalkhy/mne_data/MNE-sample-data/subjects/fsaverage/label/lh.aparc.annot
Reading labels from parcellation...
   read 34 labels from /Users/zacariabalkhy/mne_data/MNE-sample-data/subjects/fsaverage/label/rh.aparc.annot
Reading labels from parcellation...
   read 78 labels from /Users/zacariabalkhy/mne_data/MNE-sample-data/subjects/fsaverage/label/lh.aparc.a2005s.annot
Reading labels from parcellation...
   read 78 labels from /Users/zacariabalkhy/mne_data/MNE-sample-data/subjects/fsaverage/label/rh.aparc.a2005s.annot
Reading labels from parcellation...
   read 181 labels from /Users/zacariabalkhy/mne_data/MNE-sample-data/subjects/fsaverage/label/lh.HCPMMP1.annot
Reading labels from parcellation...
   read 181 labels from /Users/zacariabalkhy/mne_data/MNE-sample-data/subjects/fsaverage/label/rh.HCPMMP1.annot
Reading labels from parcellation...
   rea

We now find that the Net which was trained on large simulations always tends to find large sources - even when confronted with data in which small sources were active. 

Conversely, the Net which was trained on simulations that contain small sources finds sparse sources when confronted with data containing large-source activity.

This demonstrates that our simulation settings function like priors. Further, it emphasizes the importance to state your priors and to motivate your choice.

In many cases we can't make a choice and we want to make as few assumptions into our models as possible. In this case we propose that you use large ranges in your settings to maximize the diversity of your training data.

A sample of a diverse setting is given in the next cell:

In [9]:
settings = {
    'number_of_sources': (1, 20),  # The range of simultaneously active sources.
    'extents': (1, 50),  # The range of source diameters in mm 
    'amplitudes': (1, 100),  # Defines the range of amplitudes (in arbitrary units)
    'shapes': 'both',  # Simulate both gaussian-shaped and flat sources
    'beta': (0, 3),  # Defines the distribution of the noise in terms of 1/f**beta
}